In [4]:
import cv2
import mediapipe as mp
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

In [5]:
# Inicializando o MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Pasta com os vídeos
pasta_videos = './pasta2'

# Listas para armazenar os dados e os rótulos
dados = []
rotulos = []

In [6]:
# Dicionário para armazenar as coordenadas por palavra
coordenadas_por_palavra = {}

for subpasta in os.listdir(pasta_videos):
    for video in os.listdir(os.path.join(pasta_videos, subpasta)):
        # Lendo o vídeo
        cap = cv2.VideoCapture(os.path.join(pasta_videos, subpasta, video))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Convertendo a cor da imagem
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Processando a imagem
            result = hands.process(image)
            
            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    # Extraindo as coordenadas
                    coords = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]).flatten())
                    
                    # Adicionando as coordenadas à lista de dados
                    dados.append(coords)
                    # Adicionando o rótulo à lista de rótulos
                    rotulos.append(subpasta)
                    
                    # Armazenando as coordenadas por palavra
                    if subpasta not in coordenadas_por_palavra:
                        coordenadas_por_palavra[subpasta] = []
                    coordenadas_por_palavra[subpasta].append(coords)

# Selecionando as coordenadas mais frequentes por palavra (média das coordenadas)
coordenadas_selecionadas = {}
for palavra, coords_lista in coordenadas_por_palavra.items():
    coordenadas_selecionadas[palavra] = np.mean(coords_lista, axis=0)

# Convertendo as listas em arrays numpy
dados = np.array(dados)
rotulos = np.array(rotulos)

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(dados, rotulos, test_size=0.2, random_state=42)

# Treinando o modelo com as coordenadas selecionadas
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Avaliando o modelo
score = modelo.score(X_test, y_test)
print(f'Acurácia do modelo: {score*100:.2f}%')

# Salvando o modelo
joblib.dump(modelo, 'modelo.pkl')

c:\Users\danil\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Acurácia do modelo: 97.29%


['modelo.pkl']